In [1]:
import sys

In [2]:
MODEL_PATH = '../models'
if MODEL_PATH not in sys.path:
    sys.path.append(MODEL_PATH)

sys.path.append('../')

In [3]:
import math
import tensorflow as tf
from common.tf_layer_utils import *
from common.tf_train_utils import get_activation_fn
import os

In [4]:
from common.argparse_utils import *
parser = get_parser()

general_arg = add_argument_group('General', parser)
general_arg.add_argument('--num_threads', type=int, default=8,
                        help='the number of threads (for dataset)')

io_arg = add_argument_group('In/Out', parser)
io_arg.add_argument('--in_dir', type=str, default='../samples',
                        help='input image directory')
# io_arg.add_argument('--in_dir', type=str, default='./release/outdoor_examples/images/sacre_coeur/dense/images',
#                         help='input image directory')
io_arg.add_argument('--out_dir', type=str, default='../dump_feats',
                        help='where to save keypoints')
io_arg.add_argument('--full_output', type=str2bool, default=True,
                        help='dump keypoint image')

model_arg = add_argument_group('Model', parser)
model_arg.add_argument('--model', type=str, default='../release/models/outdoor/',
                        help='model file or directory')
model_arg.add_argument('--top_k', type=int, default=500,
                        help='number of keypoints')
model_arg.add_argument('--max_longer_edge', type=int, default=640,
                        help='resize image (do nothing if max_longer_edge <= 0)')

tmp_config, unparsed = get_config(parser)


In [5]:
import pickle

In [6]:
# restore other hyperparams to build model
if os.path.isdir(tmp_config.model):
    config_path = os.path.join(tmp_config.model, 'config.pkl')
else:
    config_path = os.path.join(os.path.dirname(tmp_config.model), 'config.pkl')
try:
    with open(config_path, 'rb') as f:
        config = pickle.load(f)
except:
    raise ValueError('Fail to open {}'.format(config_path))

In [7]:
    for attr, dst_val in sorted(vars(tmp_config).items()):
        if hasattr(config, attr):
            src_val = getattr(config, attr)
            if src_val != dst_val:
                setattr(config, attr, dst_val)
        else:
            setattr(config, attr, dst_val)

In [8]:
import tqdm
from imageio import imread, imsave
import cv2
tqdm

<module 'tqdm' from '/home/wang/.conda/envs/py3-tf/lib/python3.5/site-packages/tqdm/__init__.py'>

In [9]:
img_paths = [x.path for x in os.scandir(config.in_dir) if x.name.endswith('.jpg') or x.name.endswith('.png')]
len(img_paths)

10

In [10]:
photo = imread(img_paths[0])
height, width = photo.shape[:2]
longer_edge = max(height, width)
if config.max_longer_edge > 0 and longer_edge > config.max_longer_edge:
    if height > width:
        new_height = config.max_longer_edge
        new_width = int(width * config.max_longer_edge / height)
    else:
        new_height = int(height * config.max_longer_edge / width)
        new_width = config.max_longer_edge
    photo = cv2.resize(photo, (new_width, new_height))
    height, width = photo.shape[:2]
rgb = photo.copy()
if photo.ndim == 3 and photo.shape[-1] == 3:
    photo = cv2.cvtColor(photo, cv2.COLOR_RGB2GRAY)
photo = photo[None,...,None].astype(np.float32) / 255.0 # normalize 0-1
assert photo.ndim == 4 # [1,H,W,1]

photo.shape

(1, 477, 640, 1)

In [11]:
photo.shape

(1, 477, 640, 1)

In [12]:
wshape = [3, 3,1, 64]
bshape = 64
initializer = tf.contrib.layers.xavier_initializer()
bshape = [bshape]
W = tf.get_variable('weights', wshape, initializer=initializer, dtype=tf.float32)
b = tf.get_variable('biases', bshape, initializer=tf.zeros_initializer(), dtype=tf.float32)

In [13]:
x=tf.placeholder('float32',[1,None,None,1])
strides = [1, 2, 2, 1]
curr= tf.nn.conv2d(x, W,strides,padding='SAME')
add_bias=tf.nn.bias_add(curr, b)

init_op=tf.global_variables_initializer()
with tf.Session() as sess:
    init_op.run()
    M_conv=sess.run(add_bias,feed_dict={x:photo})

In [14]:
M_conv.shape

(1, 239, 320, 64)

下面是ｄｅｔｅｃｔｏｒ

In [15]:
pad_total = 5 - 1
pad_beg = pad_total // 2
pad_end = pad_total - pad_beg

padded_inputs = tf.pad(photo, [[0, 0], [pad_beg, pad_end],
                            [pad_beg, pad_end], [0, 0]])
padded_inputs

<tf.Tensor 'Pad:0' shape=(1, 481, 644, 1) dtype=float32>

In [21]:
wshape = [5, 5,1, 16]
bshape = 16
initializer2 = tf.contrib.layers.xavier_initializer()
bshape = [bshape]
W1 = tf.get_variable('weisdas', wshape, initializer=initializer2, dtype=tf.float32)
b1 = tf.get_variable('bassad', bshape, initializer=tf.zeros_initializer(), dtype=tf.float32)

x=tf.placeholder('float32',[1,None,None,1])
strides = [1, 1, 1, 1]
curr= tf.nn.conv2d(x, W1,strides,padding='SAME')
add_bias=tf.nn.bias_add(curr, b1)

In [23]:
init_op=tf.global_variables_initializer()
with tf.Session() as sess:
    init_op.run()
    M_conv2=sess.run(add_bias,feed_dict={x:photo})

In [24]:
M_conv2.shape

(1, 477, 640, 16)

In [131]:
base_height_f

<tf.Tensor 'strided_slice:0' shape=() dtype=int32>

In [1]:
import tensorflow as tf
input_data = tf.constant([[1.0,2,3],[4.0,5,6],[7.0,8,9]])

output = tf.nn.l2_normalize(input_data, dim = -1)
with tf.Session() as sess:
    print(sess.run(input_data))
    print(sess.run(output))

Instructions for updating:
dim is deprecated, use axis instead
[[1. 2. 3.]
 [4. 5. 6.]
 [7. 8. 9.]]
[[0.26726124 0.5345225  0.8017837 ]
 [0.45584232 0.5698029  0.6837635 ]
 [0.5025707  0.5743665  0.64616233]]


In [5]:
tf.random_uniform([2])

<tf.Tensor 'random_uniform:0' shape=(2,) dtype=float32>

In [ ]:
input_data = tf.random_uniform([2])

output = tf.nn.l2_normalize(input_data, dim = -1)
with tf.Session() as sess:
    print(sess.run(input_data))
    print(sess.run(output))